# **SVM (Support Vector Machines) — Módulo 3, Notebook 4/6**

---

## Índice

1. [Introdução ao SVM](#introducao)
2. [Como o SVM Funciona?](#como-funciona)
3. [Intuição Prática](#intuicao)
4. [Kernel Trick: Além do Linear](#kernel-trick)
5. [Formalização Matemática](#formalizacao)
6. [Implementação com Scikit-Learn](#sklearn)
7. [Vantagens e Desvantagens](#vantagens-desvantagens)

---

<a id='introducao'></a>
## **Introdução ao SVM**

Support Vector Machines (SVM) são algoritmos de aprendizado supervisionado utilizados para tarefas de classificação e regressão. O objetivo principal do SVM é encontrar o hiperplano ótimo que separa diferentes classes de dados, maximizando a margem entre elas.

Enquanto existem infinitas fronteiras de decisão possíveis que podem separar duas classes, o SVM busca aquela que oferece a maior distância entre os pontos mais próximos de cada classe - uma propriedade que confere maior robustez ao modelo.

**Comparação com outros algoritmos de classificação:**

- **KNN**: Baseado em proximidade, armazena todos os dados de treinamento
- **Naive Bayes**: Abordagem probabilística com premissa de independência entre features
- **Regressão Logística**: Modelo linear que estima probabilidades através da função logística
- **SVM**: Encontra hiperplano ótimo que maximiza a margem entre classes

**Características principais:**

- **Maximização de margem**: Otimiza a separação entre classes
- **Kernel trick**: Permite resolver problemas não-lineares através de transformações implícitas
- **Alta dimensionalidade**: Eficaz mesmo quando o número de features excede o número de amostras
- **Eficiência**: Decisão baseada apenas nos vetores de suporte

**Conteúdo do notebook:**

1. Como o SVM funciona (hiperplanos e margens)
2. Intuição prática (classificação de flores Iris)
3. Kernel trick (tratamento de não-linearidade)
4. Formalização matemática (otimização e teoria)
5. Implementação com Scikit-Learn (aplicação prática)

<a id='como-funciona'></a>
## **Como o SVM Funciona?**

O SVM resolve o problema de classificação através de três etapas fundamentais:

---

### **Etapa 1: Identificação do Hiperplano Separador**

O primeiro passo consiste em identificar um hiperplano que separa as classes:
- Em **2D**: uma linha
- Em **3D**: um plano  
- Em **N dimensões**: um hiperplano

Existem infinitas opções de hiperplanos capazes de separar os dados.

<img src="https://uk.mathworks.com/discovery/support-vector-machine/_jcr_content/mainParsys/band_1231704498_copy/mainParsys/lockedsubnav/mainParsys/columns/4d6875cb-8556-43eb-9393-53bcec9e3682/columns/29ee2f91-358a-4aca-8f04-e0f536feb8f2/image_2128876021_cop.adapt.full.medium.jpg/1759842383498.jpg" width="500">

---

### **Etapa 2: Maximização da Margem**

O SVM seleciona o hiperplano que maximiza a margem:

$$\text{Margem} = \text{distância mínima entre o hiperplano e os pontos mais próximos de cada classe}$$

**Razões para maximização da margem:**

- Maior robustez na generalização para novos dados
- Redução do risco de overfitting
- Melhoria na capacidade de generalização do modelo

---

### **Etapa 3: Definição dos Vetores de Suporte**

Os pontos mais próximos ao hiperplano são denominados vetores de suporte (support vectors).

**Propriedades dos vetores de suporte:**

- São os únicos pontos que determinam a posição do hiperplano
- A remoção de outros pontos não altera a fronteira de decisão
- Definem completamente a solução do problema de otimização

---

### **Tratamento de Dados Não-Linearmente Separáveis**

Quando os dados não podem ser separados por um hiperplano linear, o SVM utiliza o kernel trick:

1. Mapeia os dados para um espaço de maior dimensão
2. Nesse novo espaço, os dados tornam-se linearmente separáveis
3. Encontra o hiperplano ótimo no espaço transformado

**Kernels mais utilizados:**

- **Linear**: Dados já linearmente separáveis
- **Polinomial**: Relações polinomiais entre features
- **RBF (Radial Basis Function)**: Fronteiras complexas e não-lineares (mais utilizado)
- **Sigmoid**: Casos específicos

---

<a id='intuicao'></a>
## **Intuição Prática**

Para ilustrar o funcionamento do SVM, utilizaremos o dataset Iris para classificação.

### **Contexto do Problema**

Dadas as medidas de uma flor íris:
- Comprimento da pétala (cm)
- Largura da pétala (cm)

Objetivo: Classificar a espécie entre:
- Setosa
- Versicolor
- Virginica

### **Análise Exploratória**

Primeiro, visualizaremos a distribuição das flores utilizando comprimento e largura da pétala.

In [ ]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, make_circles
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
# =================================================
# 1. Análise Exploratória
# =================================================

iris = load_iris()
X = iris.data[:, [2, 3]] 
y = iris.target

print(f"Iris data shape: {X.shape}")
print(f"\n Classes:")
for i, name in enumerate(iris.target_names):
    count = np.sum(y == i)
    print(f" {i}: {name} ({count} samples)")

# Visualização dos dados
fig, ax = plt.subplots(figsize=(12, 8))

color = ['red', 'blue', 'green']
marker = ['o', 's', '^']

for i, name in enumerate(iris.target_names):
    indices = y == i
    ax.scatter(X[indices, 0], X[indices, 1], 
    c=color[i], marker=marker[i], s=100, alpha=0.7, 
    edgecolors='k', linewidth=1.5, label=name.capitalize())

ax.set_xlabel(iris.feature_names[2].capitalize(), fontsize=12, fontweight='bold')
ax.set_ylabel(iris.feature_names[3].capitalize(), fontsize=12, fontweight='bold')
ax.set_title('Iris Dataset - Sepal Length vs Sepal Width', fontsize=14, fontweight='bold', pad=20)
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# =================================================
# Visualização: Fronteiras de Decisão
# =================================================

# Transformar em problema binário para facilitar visualização
# Setosa (classe 0) vs Não-Setosa (classes 1 e 2)
y_binary = (y != 0).astype(int)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('SVM: Encontrando a Fronteira Ótima', fontsize=16, fontweight='bold', y=1.02)

# ===== GRÁFICO 1: Comparando Fronteiras =====
ax1 = axes[0]

setosa_mask = y_binary == 0
not_setosa_mask = y_binary == 1

ax1.scatter(X[setosa_mask, 0], X[setosa_mask, 1],
            c='red', s=100, alpha=0.7, edgecolors='k', linewidth=1.5,
            label='Setosa', marker='o')

ax1.scatter(X[not_setosa_mask, 0], X[not_setosa_mask, 1],
            c='blue', s=100, alpha=0.7, edgecolors='k', linewidth=1.5,
            label='Não-Setosa', marker='s')

x_line = np.linspace(0, 7, 100)

# Fronteira ruim (muito próxima de uma classe)
y_line1 = -0.3 * x_line + 1
ax1.plot(x_line, y_line1, 'gray', linestyle='--', linewidth=2, alpha=0.5, label='Fronteira Ruim')

# Fronteira OK (separa, mas não maximiza margem)
y_line2 = 0.02 * x_line + 0.7
ax1.plot(x_line, y_line2, 'orange', linestyle='--', linewidth=2, alpha=0.7, label='Fronteira OK')

# Fronteira SVM (maximiza margem)
y_line3 = -0.3 * x_line + 1.5
ax1.plot(x_line, y_line3, 'green', linestyle='-', linewidth=3, label='Fronteira SVM (ótima)')

ax1.set_xlabel('Comprimento da Pétala (cm)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Largura da Pétala (cm)', fontsize=11, fontweight='bold')
ax1.set_title('Comparando Fronteiras de Decisão', fontsize=12, fontweight='bold', pad=15)
ax1.legend(fontsize=9, loc='upper left')
ax1.set_xlim(0, 7)
ax1.set_ylim(0, 2.8)
ax1.grid(True, alpha=0.3)

# ===== GRÁFICO 2: Margens e Hiperplanos =====
ax2 = axes[1]

ax2.scatter(X[setosa_mask, 0], X[setosa_mask, 1],
            c='red', s=100, alpha=0.7, edgecolors='k', linewidth=1.5,
            label='Setosa', marker='o')

ax2.scatter(X[not_setosa_mask, 0], X[not_setosa_mask, 1],
            c='blue', s=100, alpha=0.7, edgecolors='k', linewidth=1.5,
            label='Não-Setosa', marker='s')

y_hyperplane = -0.3 * x_line + 1.5
ax2.plot(x_line, y_hyperplane, 'green', linestyle='-', linewidth=3,
         label='Hiperplano de Decisão')

# Desenhar margens
margin = 0.4
y_margin1 = y_hyperplane + margin
y_margin2 = y_hyperplane - margin

ax2.plot(x_line, y_margin1, 'green', linestyle=':', linewidth=2, alpha=0.6,
         label='Margem Superior')
ax2.plot(x_line, y_margin2, 'green', linestyle=':', linewidth=2, alpha=0.6,
         label='Margem Inferior')

# Preencher área da margem
ax2.fill_between(x_line, y_margin1, y_margin2, color='green', alpha=0.1)

ax2.set_xlabel('Comprimento da Pétala (cm)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Largura da Pétala (cm)', fontsize=11, fontweight='bold')
ax2.set_title('Margens Maximizadas', fontsize=12, fontweight='bold', pad=15)
ax2.legend(fontsize=9, loc='upper left')
ax2.set_xlim(0, 7)
ax2.set_ylim(0, 2.8)
ax2.grid(True, alpha=0.3)

# ===== GRÁFICO 3: Support Vectors =====
ax3 = axes[2]

ax3.scatter(X[setosa_mask, 0], X[setosa_mask, 1],
            c='red', s=100, alpha=0.3, edgecolors='k', linewidth=1.5,
            label='Setosa (outros)', marker='o')

ax3.scatter(X[not_setosa_mask, 0], X[not_setosa_mask, 1],
            c='blue', s=100, alpha=0.3, edgecolors='k', linewidth=1.5,
            label='Não-Setosa (outros)', marker='s')

# Destacar Support Vectors (pontos críticos)
support_vectors_setosa = np.array([[1.4, 0.2], [1.9, 0.4]])
support_vectors_not_setosa = np.array([[3.0, 1.1], [4.0, 1.3]])

ax3.scatter(support_vectors_setosa[:, 0], support_vectors_setosa[:, 1],
            c='red', s=300, alpha=1.0, edgecolors='yellow', linewidth=4,
            label='Support Vectors Setosa', marker='o')

ax3.scatter(support_vectors_not_setosa[:, 0], support_vectors_not_setosa[:, 1],
            c='blue', s=300, alpha=1.0, edgecolors='yellow', linewidth=4,
            label='Support Vectors Não-Setosa', marker='s')

ax3.plot(x_line, y_hyperplane, 'green', linestyle='-', linewidth=3, label='Hiperplano')
ax3.plot(x_line, y_margin1, 'green', linestyle=':', linewidth=2, alpha=0.6)
ax3.plot(x_line, y_margin2, 'green', linestyle=':', linewidth=2, alpha=0.6)
ax3.fill_between(x_line, y_margin1, y_margin2, color='green', alpha=0.1)

ax3.set_xlabel('Comprimento da Pétala (cm)', fontsize=11, fontweight='bold')
ax3.set_ylabel('Largura da Pétala (cm)', fontsize=11, fontweight='bold')
ax3.set_title('Support Vectors (Pontos Críticos)', fontsize=12, fontweight='bold', pad=15)
ax3.legend(fontsize=9, loc='upper left')
ax3.set_xlim(0, 7)
ax3.set_ylim(0, 2.8)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservações:")
print("   • A fronteira SVM maximiza a distância entre as classes")
print("   • Os Support Vectors (destacados) são os únicos pontos que importam")
print("   • A remoção de outros pontos não altera a fronteira")

<a id='kernel-trick'></a>
## **Kernel Trick: Tratamento de Não-Linearidade**

### **Problema: Dados Não-Linearmente Separáveis**

No exemplo anterior, conseguimos separar a classe Setosa das demais com uma fronteira linear. Porém, muitos problemas não apresentam separação linear.

Considere dados dispostos em círculos concêntricos:
- Classe Vermelha: Círculo interior
- Classe Azul: Círculo exterior

Nenhuma linha reta consegue separar essas classes adequadamente.

---

### **Solução: Kernel Trick**

O kernel trick resolve este problema através de:

**Passo 1:** Mapeamento implícito dos dados para espaço de maior dimensão  
**Passo 2:** Nesse novo espaço, os dados tornam-se linearmente separáveis  
**Passo 3:** Identificação do hiperplano ótimo no espaço transformado  
**Passo 4:** Projeção da fronteira de volta ao espaço original

---

### **Funções Kernel**

| Kernel | Equação | Aplicação |
|--------|---------|-------------|
| **Linear** | $K(x, x') = x \cdot x'$ | Dados linearmente separáveis |
| **Polinomial** | $K(x, x') = (x \cdot x' + c)^d$ | Relações polinomiais |
| **RBF (Gaussian)** | $K(x, x') = e^{-\gamma \|x - x'\|^2}$ | Fronteiras complexas |
| **Sigmoid** | $K(x, x') = \tanh(\alpha x \cdot x' + c)$ | Casos específicos |

---

In [ ]:
# =================================================
# Demonstração: Kernel Trick com Círculos Concêntricos
# =================================================

# Criar dados em círculos concêntricos (não-linearmente separáveis)
X_circles, y_circles = make_circles(n_samples=300, noise=0.05, factor=0.5, random_state=42)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Kernel Trick: Transformando Problemas Não-Lineares', fontsize=16, fontweight='bold', y=1.02)

# ===== GRÁFICO 1: Dados Originais (2D) - NÃO linearmente separáveis =====
ax1 = axes[0]

class_0 = X_circles[y_circles == 0]
class_1 = X_circles[y_circles == 1]

ax1.scatter(class_0[:, 0], class_0[:, 1], 
            c='red', s=80, alpha=0.7, edgecolors='k', linewidth=1.5,
            label='Classe 0 (Interior)', marker='o')

ax1.scatter(class_1[:, 0], class_1[:, 1],
            c='blue', s=80, alpha=0.7, edgecolors='k', linewidth=1.5,
            label='Classe 1 (Exterior)', marker='s')

# Tentar traçar linha reta (vai falhar!)
ax1.plot([-1.5, 1.5], [0, 0], 'gray', linestyle='--', linewidth=2, 
         alpha=0.5, label='Linha Reta (não funciona)')

ax1.set_xlabel('Feature X₁', fontsize=12, fontweight='bold')
ax1.set_ylabel('Feature X₂', fontsize=12, fontweight='bold')
ax1.set_title('Espaço Original (2D)', 
              fontsize=12, fontweight='bold', pad=15)
ax1.legend(fontsize=10, loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)

# ===== GRÁFICO 2: SVM com Kernel RBF - Fronteira NÃO-LINEAR =====
ax2 = axes[1]

# Treinar SVM com kernel RBF
svm_rbf = SVC(kernel='rbf', gamma=2, C=1, random_state=42)
svm_rbf.fit(X_circles, y_circles)

# Criar grid para visualizar fronteira de decisão
xx, yy = np.meshgrid(np.linspace(-1.5, 1.5, 200),
                     np.linspace(-1.5, 1.5, 200))

Z = svm_rbf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plotar fronteira de decisão
ax2.contourf(xx, yy, Z, levels=20, cmap='RdBu', alpha=0.3)
ax2.contour(xx, yy, Z, levels=[0], linewidths=3, colors='green', 
            linestyles='-', label='Fronteira SVM (RBF)')

# Plotar pontos
ax2.scatter(class_0[:, 0], class_0[:, 1],
            c='red', s=80, alpha=0.8, edgecolors='k', linewidth=1.5,
            label='Classe 0 (Interior)', marker='o')

ax2.scatter(class_1[:, 0], class_1[:, 1],
            c='blue', s=80, alpha=0.8, edgecolors='k', linewidth=1.5,
            label='Classe 1 (Exterior)', marker='s')

# Destacar Support Vectors
sv_mask = svm_rbf.support_
ax2.scatter(X_circles[sv_mask, 0], X_circles[sv_mask, 1],
            s=200, facecolors='none', edgecolors='yellow', linewidth=3,
            label=f'Support Vectors ({len(sv_mask)})')

ax2.set_xlabel('Feature X₁', fontsize=12, fontweight='bold')
ax2.set_ylabel('Feature X₂', fontsize=12, fontweight='bold')
ax2.set_title('Com Kernel RBF',
              fontsize=12, fontweight='bold', pad=15)
ax2.legend(fontsize=10, loc='upper right')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(-1.5, 1.5)
ax2.set_ylim(-1.5, 1.5)

plt.tight_layout()
plt.show()

print(f"\nResultados do SVM com Kernel RBF:")
print(f"   • Acurácia: {svm_rbf.score(X_circles, y_circles)*100:.1f}%")
print(f"   • Support Vectors: {len(svm_rbf.support_)} de {len(X_circles)} pontos ({len(svm_rbf.support_)/len(X_circles)*100:.1f}%)")
print(f"\nO Kernel RBF transformou o problema não-linear em linear em um espaço de maior dimensão.")

<a id='formalizacao'></a>
## **Formalização Matemática**

Agora que você já entendeu a **intuição** do SVM, vamos formalizar matematicamente.

Não se assuste com as equações - elas apenas descrevem o que já vimos visualmente. Se preferir, você pode pular esta seção e ir direto para a [implementação com Scikit-Learn](#sklearn).

---

### **Objetivo do SVM**

Queremos encontrar o hiperplano que **maximiza a margem** entre as classes.

**Matematicamente:** Encontrar $\mathbf{w}$ (vetor de pesos) e $b$ (bias) que definem o hiperplano:

$$\mathbf{w} \cdot \mathbf{x} + b = 0$$

**Onde:**
- $\mathbf{w}$: Vetor perpendicular ao hiperplano (define direção)
- $b$: Deslocamento do hiperplano em relação à origem
- $\mathbf{x}$: Ponto no espaço de features

---

### **Função de Decisão**

Para classificar um novo ponto $\mathbf{x}$:

$$f(\mathbf{x}) = \text{sign}(\mathbf{w} \cdot \mathbf{x} + b)$$

$$f(\mathbf{x}) = \begin{cases}
+1 & \text{se } \mathbf{w} \cdot \mathbf{x} + b > 0 \quad \text{(Classe Positiva)} \\
-1 & \text{se } \mathbf{w} \cdot \mathbf{x} + b < 0 \quad \text{(Classe Negativa)}
\end{cases}$$

---

### **Margem**

A distância de um ponto $\mathbf{x}_i$ ao hiperplano é:

$$\text{distância} = \frac{|\mathbf{w} \cdot \mathbf{x}_i + b|}{\|\mathbf{w}\|}$$

A **margem** é a distância mínima entre o hiperplano e os pontos de cada classe:

$$\text{margem} = \frac{2}{\|\mathbf{w}\|}$$

**Objetivo:** Maximizar margem = Minimizar $\|\mathbf{w}\|$

---

### **Problema de Otimização**

Queremos:

$$\min_{\mathbf{w}, b} \frac{1}{2} \|\mathbf{w}\|^2$$

**Sujeito a:** 
$$y_i (\mathbf{w} \cdot \mathbf{x}_i + b) \geq 1 \quad \forall i$$

Onde $y_i \in \{-1, +1\}$ é o rótulo verdadeiro do ponto $\mathbf{x}_i$.

---

### **Soft Margin (Margem Flexível)**

Na prática, dados podem ter **overlap** (não são perfeitamente separáveis). Usamos **variáveis de folga** $\xi_i$ (slack variables):

$$\min_{\mathbf{w}, b, \xi} \frac{1}{2} \|\mathbf{w}\|^2 + C \sum_{i=1}^{n} \xi_i$$

**Sujeito a:**
$$y_i (\mathbf{w} \cdot \mathbf{x}_i + b) \geq 1 - \xi_i \quad \text{e} \quad \xi_i \geq 0$$

**Parâmetro C:**
- **C alto**: Penaliza muito erros → margem menor, menos erros
- **C baixo**: Tolera erros → margem maior, mais erros permitidos

---

### **Kernel Trick (Formalização)**

Para problemas não-lineares, mapeamos dados para espaço de maior dimensão usando função kernel $K(\mathbf{x}_i, \mathbf{x}_j)$:

$$f(\mathbf{x}) = \text{sign}\left(\sum_{i=1}^{n} \alpha_i y_i K(\mathbf{x}_i, \mathbf{x}) + b\right)$$

**Kernels comuns:**
- **Linear**: $K(\mathbf{x}_i, \mathbf{x}_j) = \mathbf{x}_i \cdot \mathbf{x}_j$
- **Polinomial**: $K(\mathbf{x}_i, \mathbf{x}_j) = (\mathbf{x}_i \cdot \mathbf{x}_j + c)^d$
- **RBF**: $K(\mathbf{x}_i, \mathbf{x}_j) = \exp(-\gamma \|\mathbf{x}_i - \mathbf{x}_j\|^2)$

**Parâmetro γ (gamma) no RBF:**
- **γ alto**: Influência local (risco de overfitting)
- **γ baixo**: Influência ampla (risco de underfitting)

---

<a id='sklearn'></a>
## **Implementação com Scikit-Learn**

Nesta seção, aplicaremos o SVM utilizando a biblioteca Scikit-Learn no dataset Titanic para prever a sobrevivência de passageiros.

### **Problema**

Dadas informações de um passageiro:
- **Pclass**: Classe do bilhete (1ª, 2ª ou 3ª)
- **Sex**: Sexo (masculino/feminino)
- **Age**: Idade

**Objetivo:** Prever sobrevivência (0 = Não sobreviveu, 1 = Sobreviveu)

---

### **Principais Classes do Scikit-Learn**

| Classe | Descrição |
|--------|-----------|
| `SVC` | Support Vector Classification - Classificação |
| `SVR` | Support Vector Regression - Regressão |
| `LinearSVC` | SVM linear otimizado (mais rápido que SVC com kernel='linear') |

---

### **Parâmetros Principais do `SVC`**

| Parâmetro | Valores | Descrição |
|-----------|---------|-----------|
| **kernel** | `'linear'`, `'poly'`, `'rbf'`, `'sigmoid'` | Tipo de kernel a utilizar |
| **C** | float (default=1.0) | Parâmetro de regularização (C alto = margem menor, menos erros permitidos) |
| **gamma** | `'scale'`, `'auto'`, float | Coeficiente do kernel RBF/poly (gamma alto = influência local) |
| **degree** | int (default=3) | Grau do kernel polinomial |

---

In [ ]:
# =================================================
# Carregando Dataset Titanic
# =================================================

# Carregar dataset
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "yasserh/titanic-dataset",
  "Titanic-Dataset.csv",
  pandas_kwargs={"encoding": "latin", "usecols": [1, 2, 4, 5]}
)

# Preparar os dados
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Age'] = df['Age'].fillna(df['Age'].mean())

X = df[['Pclass', 'Sex', 'Age']].values
y = df['Survived'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# =================================================
# 1. Básico - SVM Linear

# Ideia:
# - Normalizar features (MUITO IMPORTANTE para SVM)
# - Treinar o modelo
# =================================================

# Normalizar os dados (crucial para SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVM básico com kernel linear
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred = svm.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Linear - Acurácia: {accuracy*100:.2f}%")

# Informações sobre o modelo
print(f"Número de Support Vectors: {svm.n_support_}")
print(f"Total de Support Vectors: {len(svm.support_)}")
print(f"Proporção de Support Vectors: {len(svm.support_)/len(X_train)*100:.1f}%")

In [ ]:
# =================================================
# 2. Testando Diferentes Kernels

# Kernels disponíveis no sklearn:
# - 'linear': para dados linearmente separáveis
# - 'poly': kernel polinomial
# - 'rbf': Radial Basis Function (Gaussiano) - default
# - 'sigmoid': kernel sigmoide
# =================================================

kernels = ['linear', 'poly', 'rbf', 'sigmoid']
results = []

for kernel in kernels:
    # Treinar SVM com kernel específico
    svm = SVC(kernel=kernel, random_state=42)
    svm.fit(X_train_scaled, y_train)
    
    # Avaliar
    y_pred = svm.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    results.append((kernel, accuracy, len(svm.support_)))
    print(f"Kernel {kernel:8}: Acurácia = {accuracy*100:5.2f}%, Support Vectors = {len(svm.support_):3d}")

print(f"\nMelhor kernel: {max(results, key=lambda x: x[1])[0]}")

In [ ]:
# =================================================
# 3. Parâmetros Importantes do SVM

# C (default=1.0): Parâmetro de regularização
# - C alto: Margem menor, menos tolerância a erros (pode causar overfitting)
# - C baixo: Margem maior, mais tolerância a erros (pode causar underfitting)

# gamma (para kernels RBF, poly, sigmoid): Define a influência de cada exemplo
# - gamma alto: Influência apenas de pontos próximos (pode causar overfitting)  
# - gamma baixo: Influência mais ampla (pode causar underfitting)
# - 'scale' (default): 1 / (n_features * X.var())
# - 'auto': 1 / n_features

# degree (para kernel poly): Grau do polinômio

# kernel: Tipo de kernel a ser usado
# =================================================

# Testar diferentes valores de C
C_values = [0.1, 1, 10, 100]
gamma_values = ['scale', 'auto', 0.001, 0.01, 0.1, 1]

print("Testando parâmetro C (com kernel RBF):")
for C in C_values:
    svm = SVC(kernel='rbf', C=C, random_state=42)
    svm.fit(X_train_scaled, y_train)
    accuracy = svm.score(X_test_scaled, y_test)
    print(f"C = {C:5.1f}: Acurácia = {accuracy*100:5.2f}%, Support Vectors = {len(svm.support_):3d}")

print(f"\nTestando parâmetro gamma (com kernel RBF, C=1):")
for gamma in gamma_values:
    svm = SVC(kernel='rbf', C=1, gamma=gamma, random_state=42)
    svm.fit(X_train_scaled, y_train)
    accuracy = svm.score(X_test_scaled, y_test)
    print(f"gamma = {str(gamma):5}: Acurácia = {accuracy*100:5.2f}%, Support Vectors = {len(svm.support_):3d}")

In [ ]:
# =================================================
# 4. Otimização de Hiperparâmetros com Grid Search
# =================================================

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Definir espaço de busca
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear']
}

# Grid search com validação cruzada
svm_grid = SVC(random_state=42)
grid_search = GridSearchCV(svm_grid, param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train_scaled, y_train)

# Melhores parâmetros
print(f"Melhores parâmetros: {grid_search.best_params_}")
print(f"Melhor score CV: {grid_search.best_score_*100:.2f}%")

# Avaliar no conjunto de teste
best_svm = grid_search.best_estimator_
y_pred_best = best_svm.predict(X_test_scaled)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Acurácia no teste: {accuracy_best*100:.2f}%")

# Relatório detalhado
print(f"\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_best))

In [ ]:
# =================================================
# 5. Interpretação dos Resultados

# Vamos analisar as predições para alguns casos específicos
# =================================================
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Casos de teste específicos
test_cases = [
    [1, 1, 25],  # 1ª classe, feminino, 25 anos - deveria sobreviver
    [3, 0, 22],  # 3ª classe, masculino, 22 anos - provavelmente não sobrevive
    [2, 1, 35],  # 2ª classe, feminino, 35 anos - provavelmente sobrevive
    [3, 1, 5],   # 3ª classe, feminino, 5 anos - criança, provavelmente sobrevive
]

print("Predições para casos específicos:")
print("=" * 60)

for i, case in enumerate(test_cases):
    # Normalizar o caso de teste
    case_scaled = scaler.transform([case])
    
    # Fazer predição
    prediction = best_svm.predict(case_scaled)[0]
    confidence = best_svm.decision_function(case_scaled)[0]
    
    # Mapear características
    class_names = {1: "1ª classe", 2: "2ª classe", 3: "3ª classe"}
    sex_names = {0: "Masculino", 1: "Feminino"}
    result_names = {0: "Não Sobrevive", 1: "Sobrevive"}
    
    print(f"Caso {i+1}:")
    print(f"  Classe: {class_names[case[0]]}")
    print(f"  Sexo: {sex_names[case[1]]}")
    print(f"  Idade: {case[2]} anos")
    print(f"  Predição: {result_names[prediction]}")
    print(f"  Confiança: {abs(confidence):.3f}")
    print(f"  {'*' * 40}")

# Matriz de confusão
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Não Sobrevive', 'Sobrevive'],
            yticklabels=['Não Sobrevive', 'Sobrevive'])
plt.title('Matriz de Confusão - SVM Otimizado')
plt.ylabel('Valor Real')
plt.xlabel('Predição')
plt.show()

<a id='vantagens-desvantagens'></a>
## **Vantagens e Desvantagens**

### **Vantagens**

1. **Maximização de margem**: Otimiza separação entre classes, resultando em fronteiras robustas
2. **Alta dimensionalidade**: Eficaz mesmo quando número de features excede número de amostras
3. **Eficiência computacional**: Utiliza apenas vetores de suporte, não todos os dados de treinamento
4. **Versatilidade**: Diferentes kernels permitem capturar relacionamentos complexos
5. **Robustez**: Menos propenso a overfitting em alta dimensionalidade
6. **Fundamento teórico**: Baseado em teoria de otimização e aprendizado estatístico

### **Desvantagens**

1. **Sensibilidade à escala**: Features em escalas diferentes afetam resultado - normalizar com `StandardScaler`
2. **Complexidade computacional**: $O(n^2)$ a $O(n^3)$ em datasets grandes - usar `LinearSVC` ou `SGDClassifier`
3. **Ajuste de hiperparâmetros**: Requer busca extensa para C, γ, kernel - utilizar `GridSearchCV`
4. **Probabilidades**: Não retorna probabilidades diretamente - usar `probability=True` (custo computacional)
5. **Classificação multiclasse**: SVM é binário nativamente - Sklearn implementa one-vs-one automaticamente
6. **Interpretabilidade**: Dificuldade em entender importância de features - usar `LinearSVC` com atributo `coef_`

---

### **Quando usar SVM?**

**Casos recomendados:**
- Fronteiras de decisão não-lineares complexas
- Dados de alta dimensionalidade (muitas features)
- Precisão prioritária sobre interpretabilidade
- Datasets pequenos a médios (< 10.000 amostras)
- Necessidade de robustez contra overfitting

**Casos não recomendados:**
- Necessidade de probabilidades calibradas - preferir Regressão Logística
- Datasets muito grandes (> 100.000 amostras) - preferir Naive Bayes ou Regressão Logística
- Requisitos de interpretabilidade - preferir Árvores de Decisão ou Regressão Logística
- Features em escalas muito diferentes sem possibilidade de normalização

---

### **Aplicações**

| Área | Exemplos |
|------|----------|
| Processamento de texto | Classificação de sentimentos, detecção de spam, categorização de documentos |
| Visão computacional | Reconhecimento facial, detecção de objetos, OCR |
| Bioinformática | Classificação de proteínas, predição de genes |
| Finanças | Detecção de fraude, análise de crédito |
| Medicina | Diagnóstico de doenças, classificação de tumores |

---

### **Recomendações Práticas**

1. Normalizar features com `StandardScaler` antes do treinamento
2. Iniciar com kernel RBF (funcionalidade adequada para maioria dos problemas)
3. Utilizar Grid Search para otimização de hiperparâmetros:
   ```python
   param_grid = {
       'C': [0.1, 1, 10, 100],
       'gamma': ['scale', 0.001, 0.01, 0.1, 1],
       'kernel': ['rbf', 'linear']
   }
   ```
4. Para datasets grandes, preferir `LinearSVC`
5. Para probabilidades, usar `SVC(probability=True)` considerando custo computacional adicional
6. Em problemas multiclasse, SVM utiliza estratégia one-vs-one automaticamente

---

## **Recursos Adicionais**

**Documentação Oficial:**
- [Scikit-Learn: SVM](https://scikit-learn.org/stable/modules/svm.html)
- [SVC API Reference](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
- [SVR API Reference](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)

**Tutoriais e Artigos:**
- [The Kernel Trick](https://medium.com/data-science/the-kernel-trick-c98cdbcaeb3f)
- [Understanding Support Vector Machine](https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/)
- [SVM Tutorial](https://www.datacamp.com/tutorial/svm-classification-scikit-learn-python)

**Vídeos Recomendados:**
- [StatQuest: Support Vector Machines](https://www.youtube.com/watch?v=efR1C6CvhmE)
- [SVM with Polynomial Kernel Visualization](https://www.youtube.com/watch?v=3liCbRZPrZA)

**Datasets:**
- [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)
- [Kaggle Datasets](https://www.kaggle.com/datasets)

---

## **Resumo**

Neste notebook foram abordados:

- Conceitos fundamentais do SVM (hiperplano, margem, vetores de suporte)
- Funcionamento em 3 etapas (hiperplano, maximização de margem, support vectors)
- Kernel trick para tratamento de não-linearidade
- Formalização matemática (hiperplanos, margens, otimização)
- Implementação prática com Scikit-Learn
- Vantagens e limitações do algoritmo

**Conceitos principais:**
- **Hiperplano**: Fronteira que separa classes
- **Margem**: Distância entre hiperplano e pontos mais próximos
- **Support Vectors**: Pontos críticos que definem a fronteira
- **Kernel Trick**: Mapeamento para espaço de maior dimensão
- **Parâmetro C**: Controla trade-off entre margem e erros
- **Parâmetro γ**: Controla influência de cada ponto (kernel RBF)

---

<-- [**Anterior: Regressão Logística**](03_regressao_logistica.ipynb) | [**Próximo: Árvores de Decisão**](05_arvores_classificacao.ipynb) -->